In [1]:
# 步驟 1: 匯入所需的套件
# selenium 用於自動化網頁瀏覽器操作
from selenium import webdriver
# By 用於定位網頁元素的方式（如 CSS_SELECTOR, TAG_NAME 等）
from selenium.webdriver.common.by import By
# WebDriverWait 用於等待特定條件達成（顯式等待）
from selenium.webdriver.support.ui import WebDriverWait
# expected_conditions (EC) 提供常用的等待條件
from selenium.webdriver.support import expected_conditions as EC
# Select 用於操作下拉選單（<select> 元素）
from selenium.webdriver.support.ui import Select
# pandas 用於資料處理和儲存成 CSV 格式
import pandas as pd
# time 用於控制程式執行速度（避免對伺服器造成負擔）
import time

print("所有套件已成功匯入！")


所有套件已成功匯入！


In [2]:
# 步驟 2: 設定 Chrome 瀏覽器選項並啟動
# webdriver.ChromeOptions() 創建一個 Chrome 瀏覽器的配置對象
options = webdriver.ChromeOptions()

# add_argument("--headless") 讓瀏覽器在背景執行，不會開啟視窗（節省資源）
# options.add_argument("--headless")

# add_argument("--no-sandbox") 關閉沙盒模式，在某些環境中需要此設定
options.add_argument("--no-sandbox")

# add_argument("--disable-dev-shm-usage") 解決資源限制問題
options.add_argument("--disable-dev-shm-usage")

# 以下選項幫助 headless 模式更像真實瀏覽器，避免被網站偵測

# 設定視窗大小，模擬真實瀏覽器的解析度
# 如果不設定，headless 模式的視窗可能是 800x600（看起來很可疑）
options.add_argument("--window-size=1920,1080")

# 設定 User-Agent，讓網站認為這是正常的 Chrome 瀏覽器
# 預設的 headless User-Agent 會包含 "HeadlessChrome" 字樣
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

# 禁用 Blink 引擎的自動化控制特徵
# 這個特徵會讓網站知道瀏覽器正在被 Selenium 控制
options.add_argument("--disable-blink-features=AutomationControlled")

# 排除啟用自動化擴展的開關
# 進一步隱藏 Selenium 的痕跡
options.add_experimental_option("excludeSwitches", ["enable-automation"])

# 禁用瀏覽器正在被自動化程式控制的提示
options.add_experimental_option('useAutomationExtension', False)

# webdriver.Chrome(options=options) 使用上述配置啟動 Chrome 瀏覽器
driver = webdriver.Chrome(options=options)

# 修改 navigator.webdriver 屬性，讓網站無法偵測到這是 Selenium
# 正常情況下，Selenium 會設定 navigator.webdriver = true
# 這行程式碼將它改為 undefined，就像真實瀏覽器一樣
driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
    'source': '''
        Object.defineProperty(navigator, 'webdriver', {
            get: () => undefined
        })
    '''
})

# WebDriverWait(driver, 120) 創建一個等待物件，最多等待 120 秒
# 當需要等待網頁元素載入時使用
wait = WebDriverWait(driver, 45)

print("Chrome 瀏覽器已啟動（Headless 模式 + 反偵測配置）！")


Chrome 瀏覽器已啟動（Headless 模式 + 反偵測配置）！


In [3]:
# 步驟 3: 打開 RealGM 的 NBA 球員列表頁面
# driver.get() 讓瀏覽器訪問指定的網址
# 這個網址包含 2025-26 賽季所有 NBA 球員的列表
driver.get("https://basketball.realgm.com/nba/players")


# wait.until() 等待直到指定條件達成
# EC.presence_of_element_located() 檢查元素是否出現在 DOM 中
# (By.CSS_SELECTOR, "table") 使用 CSS 選擇器尋找 <table> 元素
# 這行代碼確保頁面上的表格已經載入完成
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table")))

print("球員列表頁面已載入完成！")


球員列表頁面已載入完成！


In [4]:
# 步驟 4: 擷取所有球員的連結
# driver.find_elements() 尋找所有符合條件的元素（複數）
# By.CSS_SELECTOR 使用 CSS 選擇器定位元素
# "table tbody tr td:nth-child(1) a" 的意思是：
#   - table: 表格元素
#   - tbody: 表格主體
#   - tr: 表格的每一列
#   - td:nth-child(1): 每列的第一個儲存格（球員姓名所在欄位）
#   - a: 該儲存格中的超連結（<a> 標籤）
player_links = driver.find_elements(By.CSS_SELECTOR, "table tbody tr td:nth-child(2) a")


# 使用列表推導式（list comprehension）提取所有連結的網址
# link.get_attribute("href") 取得每個 <a> 標籤的 href 屬性（網址）
# for link in player_links 遍歷所有找到的超連結元素
links = [link.get_attribute("href") for link in player_links]

# len(links) 計算列表的長度，即球員數量
print(f"找到 {len(links)} 位球員的連結！")
# 顯示前 3 個連結作為範例
print("前 3 個球員連結範例：")
for i, link in enumerate(links[:3], 1):
    print(f"  {i}. {link}")


找到 527 位球員的連結！
前 3 個球員連結範例：
  1. https://basketball.realgm.com/player/Precious-Achiuwa/Summary/107977
  2. https://basketball.realgm.com/player/Steven-Adams/Summary/27993
  3. https://basketball.realgm.com/player/Bam-Adebayo/Summary/76939


In [ ]:
# 步驟 5: 逐一訪問每位球員的 Game Log 頁面並抓取數據（改進版）
# all_logs 是一個空列表，用來儲存所有球員的 game log 資料
all_logs = []

# 新增：用來記錄失敗原因的字典
# 這樣可以統計各種錯誤類型的發生次數
failure_reasons = {}

# 新增：成功和失敗的計數器
success_count = 0
failure_count = 0

# enumerate(links) 同時取得索引（idx）和值（link）
# start=1 讓索引從 1 開始計數（方便顯示進度）
for idx, link in enumerate(links, start=1):
    # 將球員概要頁面的 URL 轉換成 Game Log 頁面的 URL
    # replace("/Summary/", "/GameLogs/") 把網址中的 "Summary" 替換成 "GameLogs"
    # 例如：.../Summary/83939 變成 .../GameLogs/83939
    game_log_url = link.replace("/Summary/", "/GameLogs/")
    
    # 新增：設定重試次數（最多重試 3 次）
    # 這樣可以處理偶發性的網路問題或頁面載入問題
    max_retries = 3
    retry_count = 0
    success = False
    
    # 使用 while 迴圈實現重試機制
    while retry_count < max_retries and not success:
        try:
            # 如果是重試，顯示重試訊息
            if retry_count > 0:
                print(f"  ⟳ 重試第 {retry_count} 次...")
            
            # 訪問該球員的 Game Log 頁面
            driver.get(game_log_url)
            
            # 等待表格出現（最多等待 45 秒，使用之前設定的 wait 物件）
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table")))
            
            # 嘗試取得球員姓名（從頁面標題或其他元素）
            # 根據 HTML 結構，球員姓名在 div.half-column-left 內的 h2 元素中
            try:
                # 找到 h2 元素
                h2_element = driver.find_element(By.CSS_SELECTOR, "div.half-column-left h2")
                # 使用 JavaScript 來取得 h2 的直接文字內容，排除 span 標籤內的文字
                # childNodes 是該元素的所有子節點
                # nodeType === Node.TEXT_NODE 檢查是否為純文字節點
                # nodeValue 取得該節點的文字值
                player_name_raw = driver.execute_script("""
                    var h2 = arguments[0];
                    var text = '';
                    for (var i = 0; i < h2.childNodes.length; i++) {
                        if (h2.childNodes[i].nodeType === Node.TEXT_NODE) {
                            text += h2.childNodes[i].nodeValue;
                        }
                    }
                    return text.trim();
                """, h2_element)
                player_name = player_name_raw.strip()
            except:
                # 如果無法取得姓名，使用預設值
                player_name = f"球員 {idx}"
            
            # 只在第一次嘗試時顯示進度訊息
            if retry_count == 0:
                print(f"正在抓取第 {idx}/{len(links)} 位球員：{player_name}")
            
            # 尋找頁面上所有的 <select> 元素（下拉選單）
            # RealGM 的 Game Log 頁面有三個選擇器：League、Season、Games
            selects = driver.find_elements(By.TAG_NAME, "select")
            
            # 確保頁面有三個選擇器
            if len(selects) >= 3:
                # 第一個選擇器：League（聯盟）- 選擇 "NBA"
                # select_by_visible_text() 會根據可見文字選擇選項
                league_select = Select(selects[0])
                try:
                    # 嘗試選擇 "NBA" 選項
                    league_select.select_by_visible_text("NBA")
                except:
                    # 如果找不到 "NBA"，嘗試使用索引 0（通常是預設選項）
                    # select_by_index() 根據選項的索引位置選擇（從 0 開始）
                    try:
                        league_select.select_by_index(0)
                    except:
                        # 如果連索引都選不到，跳過（可能已經是預設值）
                        pass
                # 改進：增加等待時間從 0.5 秒到 1.5 秒
                # 讓頁面有足夠時間完成 AJAX 請求和 DOM 更新
                time.sleep(1.5)
                
                # 第二個選擇器：Season（賽季）- 選擇 "2025-2026"
                season_select = Select(selects[1])
                try:
                    # 嘗試標準格式 "2025-2026"
                    season_select.select_by_visible_text("2025-2026")
                except:
                    # 如果找不到，嘗試短格式 "2025-26"
                    try:
                        season_select.select_by_visible_text("2025-26")
                    except:
                        # 如果還是找不到，使用預設選項
                        try:
                            season_select.select_by_index(0)
                        except:
                            pass
                # 改進：增加等待時間
                time.sleep(1.5)
                
                # 第三個選擇器：Games（比賽類型）- 選擇 "All Games"
                # 改進：增加更強健的選項選擇邏輯
                # 因為不同球員的頁面可能有不同的選項文字格式
                games_select = Select(selects[2])
                
                # possible_game_options 列出所有可能的選項文字
                # 按優先順序排列（最常見的放前面）
                possible_game_options = [
                    "All Games",  # 最常見的格式
                    "Regular Season"
                ]
                
                # game_option_selected 用來追蹤是否成功選擇了選項
                game_option_selected = False
                # 遍歷所有可能的選項文字，嘗試選擇
                for option_text in possible_game_options:
                    try:
                        games_select.select_by_visible_text(option_text)
                        # 如果成功，標記為 True 並跳出迴圈
                        game_option_selected = True
                        break
                    except:
                        # 如果失敗，繼續嘗試下一個選項
                        continue
                
                # 如果上述所有選項都找不到，嘗試使用索引 0（預設選項）
                if not game_option_selected:
                    try:
                        games_select.select_by_index(0)
                    except:
                        # 如果連索引都選不到，記錄但繼續執行
                        # 可能該球員的頁面已經預設顯示所有比賽
                        pass
                
                # 改進：增加等待時間到 2 秒
                # 對於數據量大的球員（如 LeBron），需要更多時間載入
                time.sleep(2)
                
                # 改進：等待表格完全載入
                # 使用 staleness_of 來確保舊的表格已經被移除
                # 然後再等待新的表格出現
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table tbody")))
                
                # 改進：額外等待 1 秒，確保表格內容完全渲染
                time.sleep(1)
                
                # 改進：在迴圈外重新查找所有列
                # 這樣可以避免 stale element 錯誤
                # 因為我們每次都是從最新的 DOM 中查找元素
                rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
                
                # 記錄該球員的比賽數量
                game_count = 0
                
                # 改進：使用索引遍歷，而不是直接遍歷元素
                # 這樣可以在每次迭代時重新查找元素，避免 stale element 錯誤
                for row_idx in range(len(rows)):
                    try:
                        # 改進：每次迭代都重新查找所有列
                        # 這是避免 stale element 錯誤的關鍵！
                        rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
                        
                        # 確保該列索引仍然有效（防止表格在抓取過程中改變）
                        if row_idx >= len(rows):
                            break
                        
                        # 取得當前列
                        row = rows[row_idx]
                        
                        # 尋找該列中所有的儲存格（<td> 元素）
                        cells = row.find_elements(By.TAG_NAME, "td")
                        # 提取每個儲存格的文字，並移除前後空白
                        # cell.text.strip() 取得文字並去除空白
                        cell_data = [cell.text.strip() for cell in cells]
                        
                        # 如果該列有資料（不是空列）
                        if cell_data:
                            # 在資料前面加上球員姓名和賽季資訊
                            # [player_name, "2025-2026"] + cell_data 將球員姓名和賽季名稱加到列表開頭
                            cell_data_with_info = [player_name, "2025-2026"] + cell_data
                            # append() 將這筆資料加入 all_logs 列表
                            all_logs.append(cell_data_with_info)
                            game_count += 1
                    
                    except Exception as row_error:
                        # 如果某一列抓取失敗，記錄但繼續處理下一列
                        # 這樣不會因為一列的問題而放棄整個球員的資料
                        continue
                
                print(f"  ✓ 成功抓取 {game_count} 場比賽資料")
                success = True
                success_count += 1
                
            else:
                # 如果沒有找到三個選擇器，直接抓取當前頁面的表格
                rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
                game_count = 0
                
                # 同樣使用索引遍歷
                for row_idx in range(len(rows)):
                    try:
                        rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
                        if row_idx >= len(rows):
                            break
                        row = rows[row_idx]
                        cells = row.find_elements(By.TAG_NAME, "td")
                        cell_data = [cell.text.strip() for cell in cells]
                        if cell_data:
                            # [player_name, "N/A"] 加上球員姓名，賽季標記為 N/A
                            cell_data_with_info = [player_name, "N/A"] + cell_data
                            all_logs.append(cell_data_with_info)
                            game_count += 1
                    except:
                        continue
                
                print(f"  ✓ 成功抓取 {game_count} 場比賽資料（無選擇器）")
                success = True
                success_count += 1
            
            # time.sleep(1) 暫停 1 秒，避免對伺服器造成過大負擔
            time.sleep(1)
            
        except Exception as e:
            # 增加重試計數
            retry_count += 1
            
            # 如果還有重試機會，等待一下再重試
            if retry_count < max_retries:
                # 等待時間隨重試次數遞增（1秒、2秒、3秒）
                wait_time = retry_count
                time.sleep(wait_time)
            else:
                # 如果已經重試 3 次都失敗，記錄錯誤
                # 改進：分類錯誤類型
                error_message = str(e)
                
                # 判斷錯誤類型
                if "stale element" in error_message.lower():
                    error_type = "Stale Element (元素過時)"
                elif "timeout" in error_message.lower() or "timed out" in error_message.lower():
                    error_type = "Timeout (超時)"
                elif "no such element" in error_message.lower():
                    error_type = "Element Not Found (找不到元素)"
                else:
                    error_type = "Other (其他)"
                
                # 記錄錯誤類型
                failure_reasons[error_type] = failure_reasons.get(error_type, 0) + 1
                
                # 顯示錯誤訊息（只顯示前 100 個字元，避免訊息過長）
                short_error = error_message[:100] + "..." if len(error_message) > 100 else error_message
                print(f"  ✗ 無法抓取球員 {idx} 的資料 [{error_type}]：{short_error}")
                failure_count += 1

# 顯示詳細的統計資訊
print(f"\n{'='*60}")
print(f"抓取完成！統計資訊：")
print(f"{'='*60}")
print(f"✓ 成功：{success_count} 位球員 ({success_count/len(links)*100:.1f}%)")
print(f"✗ 失敗：{failure_count} 位球員 ({failure_count/len(links)*100:.1f}%)")
print(f"📊 總共抓取了 {len(all_logs)} 筆 game log 資料")
print(f"\n失敗原因分析：")
for reason, count in sorted(failure_reasons.items(), key=lambda x: x[1], reverse=True):
    print(f"  • {reason}: {count} 次 ({count/failure_count*100:.1f}%)" if failure_count > 0 else f"  • {reason}: {count} 次")
print(f"{'='*60}")


正在抓取第 1/527 位球員：Precious Achiuwa
  ✓ 成功抓取 45 場比賽資料
正在抓取第 2/527 位球員：Steven Adams
  ✓ 成功抓取 34 場比賽資料
正在抓取第 3/527 位球員：Bam Adebayo
  ✓ 成功抓取 44 場比賽資料
正在抓取第 4/527 位球員：Ochai Agbaji
  ✓ 成功抓取 45 場比賽資料
正在抓取第 5/527 位球員：Santi Aldama
  ✓ 成功抓取 47 場比賽資料
正在抓取第 6/527 位球員：Trey Alexander
  ⟳ 重試第 1 次...
  ⟳ 重試第 2 次...
  ✗ 無法抓取球員 6 的資料 [Stale Element (元素過時)]：Message: stale element reference: stale element not found
  (Session info: chrome=144.0.7559.97); Fo...
正在抓取第 7/527 位球員：Nickeil Alexander-Walker
  ✓ 成功抓取 48 場比賽資料
正在抓取第 8/527 位球員：Grayson Allen
  ✓ 成功抓取 33 場比賽資料
正在抓取第 9/527 位球員：Jarrett Allen
  ✓ 成功抓取 39 場比賽資料
正在抓取第 10/527 位球員：Jose Alvarado
  ✓ 成功抓取 41 場比賽資料
正在抓取第 11/527 位球員：Kyle Anderson
  ✓ 成功抓取 20 場比賽資料
正在抓取第 12/527 位球員：Alex Antetokounmpo
  ⟳ 重試第 1 次...
  ⟳ 重試第 2 次...
  ✗ 無法抓取球員 12 的資料 [Stale Element (元素過時)]：Message: stale element reference: stale element not found
  (Session info: chrome=144.0.7559.97); Fo...
正在抓取第 13/527 位球員：Giannis Antetokounmpo
  ✓ 成功抓取 32 場比賽資料
正在抓取第 14/527 位球員：Thanasis Antetokoun

In [ ]:
# 步驟 6: 將抓取的資料轉換成 pandas DataFrame 並儲存
# pd.DataFrame() 將列表轉換成 pandas 的 DataFrame 格式
# columns 參數指定每一欄的名稱
# 這些欄位名稱對應 RealGM Game Log 表格的標題

df = pd.DataFrame(all_logs, columns=[
    "Player",
    "Season",
    "Date",
    "Team",
    "Opponent",
    "W/L",
    "Status",
    "Pos",
    "MIN",
    "PTS",
    "FGM",
    "FGA",
    "FG%",
    "3PM",
    "3PA",
    "3P%",
    "FTM",
    "FTA",
    "FT%",
    "ORB",
    "DRB",
    "REB",
    "AST",
    "STL",
    "BLK",
    "TOV",
    "PF",
    "FIC"
])

# df.to_csv() 將 DataFrame 儲存成 CSV 檔案
# "nba_player_game_logs.csv" 是檔案名稱
# index=False 表示不儲存索引欄（列號）
# encoding='utf-8-sig' 確保中文字元正確顯示（若有的話）
df.to_csv("nba_player_game_logs.csv", index=False, encoding='utf-8-sig')

# df.head() 顯示 DataFrame 的前 5 列資料，用來預覽結果
print("資料預覽（前 5 筆）：")
print(df.head())
print(f"\n資料已成功儲存至 nba_player_game_logs.csv")
print(f"總共有 {len(df)} 筆記錄，{len(df.columns)} 個欄位")


資料預覽（前 5 筆）：
             Player     Season       Date   Team       Opponent W/L   Status  \
0  Precious Achiuwa  2025-2026  1/25/2026  Kings        Pistons   L  Starter   
1  Precious Achiuwa  2025-2026  1/23/2026  Kings      Cavaliers   L  Starter   
2  Precious Achiuwa  2025-2026  1/21/2026  Kings        Raptors   L  Starter   
3  Precious Achiuwa  2025-2026  1/20/2026  Kings           Heat   L  Starter   
4  Precious Achiuwa  2025-2026  1/18/2026  Kings  Trail Blazers   L  Starter   

  Pos    MIN PTS  ...    FT% ORB DRB REB AST STL BLK TOV PF   FIC  
0  PF  17:21   4  ...   .000   2   2   4   1   0   0   1  1   4.0  
1  PF  20:22   4  ...   .000   3   5   8   2   0   0   1  2   6.3  
2  PF  20:43  11  ...   .000   2   6   8   1   0   0   0  1  13.5  
3  PF  22:04   6  ...  1.000   0   5   5   1   1   0   2  1   5.5  
4  PF  17:43   4  ...   .000   0   3   3   2   0   1   1  3   4.5  

[5 rows x 28 columns]

資料已成功儲存至 nba_player_game_logs.csv
總共有 14950 筆記錄，28 個欄位


In [ ]:
# 步驟 7: 關閉瀏覽器
# driver.quit() 關閉瀏覽器並結束 WebDriver 程序
# 這一步很重要，可以釋放系統資源
driver.quit()

print("瀏覽器已關閉，程式執行完畢！")


瀏覽器已關閉，程式執行完畢！
